In [3]:
from dotenv import load_dotenv
import os
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.llms import OpenAI
from serpapi import GoogleSearch

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_KEY")
print(OPENAI_API_KEY[0:6], SERPAPI_KEY[0:6])

sk-859 24ada9


In [4]:

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:



template = """
    I'll give you three inputs. These inputs will be the name of the company, 
    the country of the company, and the website company. The website of the company
    is not mandatory.
    You have to give me the products and services that the company offers as output.
    you dont need to give me nothing more than the ouput.



    input:
    IKEA Deutschland GmbH & Co. KG
    Germany
    ikea.com

    the output must be in this format, please use it:
    "Products/services": Furniture, Home decor, Kitchen and Dining;
    "Keywords":furniture, storage, lighting;
    "Company Classification":5712 (Furniture Stores) – SIC, 442110 (Furniture Stores) – NAICS
    do it yourself now.
    input:
    {name_of_company}
    {country_of_company}
    {website_of_company}

    what is the output?
"""

prompt = PromptTemplate(
    input_variables=["name_of_company", "country_of_company", "website_of_company"],
    template=template,
)

In [6]:

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({"name_of_company": "Google", 
           "country_of_company": "United States", 
           "website_of_company":"google.com"

})

In [7]:
print(output)

"Products/services": Search engine, Advertising, Cloud computing, Software;
    "Keywords": Search, Advertising, Cloud, Software, Technology;
    "Company Classification": 7370 (Computer Programming, Data Processing, and Related Services) – SIC, 511210 (Software Publishers) – NAICS.


In [ ]:
#query de busca no serpapi:
#nomedaempresa + paisdaempresa + palavrachave1 + palavrachave2 + palavrachave3

In [8]:
def parse_output(output_langchain: str) -> dict:
  """ 
  Will parse the output_langchain of the Langchain query
  
  Args:
    output(str): The output_langchain of Langchain query.

  Returns:
    result_dict(dict): the parsed output_langchain to dict
  """

  result_dict = {}

  sections = [section.strip() for section in output.split(";")]

  for section in sections:
      if section:
          header, values_str = section.split(":")
          header = header.strip('"')
          values = [value.strip() for value in values_str.strip('[]').split(",")]

          result_dict[header] = values

  return result_dict

In [9]:
parsed = parse_output(output)

In [11]:
parsed["images"] = "test"
parsed

{'Products/services': ['Search engine',
  'Advertising',
  'Cloud computing',
  'Software'],
 'Keywords': ['Search', 'Advertising', 'Cloud', 'Software', 'Technology'],
 'Company Classification': ['7370 (Computer Programming',
  'Data Processing',
  'and Related Services) – SIC',
  '511210 (Software Publishers) – NAICS.'],
 'images': 'test'}

In [8]:
parsed["Products/services"]

['Search engine', 'Advertising', 'Cloud computing', 'Software']

# Teste com SerpAPI

In [9]:
#!pip install google-search-results


In [11]:
search = GoogleSearch({
    "q": "Polar bear",
    "engine": "google_images",
    "location": "Austin, Texas",
    "api_key": SERPAPI_KEY
})

In [12]:
search

In [ ]:
response = search.get_dict()
print(response)

In [16]:
response.keys()

dict_keys(['search_metadata', 'search_parameters', 'search_information', 'suggested_searches', 'images_results', 'related_searches'])

In [24]:
imgs = [r["original"] for r in response["images_results"][:5]]
imgs

['https://upload.wikimedia.org/wikipedia/commons/6/66/Polar_Bear_-_Alaska_%28cropped%29.jpg',
 'https://i.natgeofe.com/k/55256f3f-2cf1-4b93-9d95-a13b0faa30a6/Mom-and-Babies_Polar-Bear_KIDS_0223-crop_3x2.jpg',
 'https://files.worldwildlife.org/wwfcmsprod/images/Polar_bear_on_ice_in_Svalbard_Norway_WW294883/story_full_width/42ny6cwj8t_Polar_bear_on_ice_in_Svalbard_Norway_WW294883.jpg',
 'https://good-nature-blog-uploads.s3.amazonaws.com/uploads/2022/07/Polar-Bear-playing-in-the-snow-by-Eddy-Savage-1280x640.png',
 'https://optimise2.assets-servd.host/maniacal-finch/production/animals/polar-bear-01-01.jpg?w=1200&auto=compress%2Cformat&fit=crop&dm=1658950229&s=92bb7b274a3ab178ae54ddf5b186306b']

In [26]:
def google_search(query: str) -> list:
    """
    Will query google for images based in the output of OpenAIAPI
    
    Args:
        query(str): Formated query using the output openaiAPI
    
    Results:
        imgs(list): List with URLs for images
    """
    search = GoogleSearch({
    "q": query,
    "engine": "google_images",
    "location": "Austin, Texas",
    "api_key": SERPAPI_KEY
    })
    response = search.get_dict()
    imgs = [r["original"] for r in response["images_results"][:5]]

    return imgs

In [27]:
tst = google_search("Polar bear")
tst

['https://upload.wikimedia.org/wikipedia/commons/6/66/Polar_Bear_-_Alaska_%28cropped%29.jpg',
 'https://i.natgeofe.com/k/55256f3f-2cf1-4b93-9d95-a13b0faa30a6/Mom-and-Babies_Polar-Bear_KIDS_0223-crop_3x2.jpg',
 'https://files.worldwildlife.org/wwfcmsprod/images/Polar_bear_on_ice_in_Svalbard_Norway_WW294883/story_full_width/42ny6cwj8t_Polar_bear_on_ice_in_Svalbard_Norway_WW294883.jpg',
 'https://good-nature-blog-uploads.s3.amazonaws.com/uploads/2022/07/Polar-Bear-playing-in-the-snow-by-Eddy-Savage-1280x640.png',
 'https://optimise2.assets-servd.host/maniacal-finch/production/animals/polar-bear-01-01.jpg?w=1200&auto=compress%2Cformat&fit=crop&dm=1658950229&s=92bb7b274a3ab178ae54ddf5b186306b']

In [28]:
#prompt creation -> openai query -> google query creation -> google search

def gpt_call(name: str, country: str, website: str = None) -> dict:
    """
    Will call gpt-3.5-turbo for querying informations about a company.

    Args:
        name(str): Name of the company
        country(str): Country of the company
        website(str): Website of the company

    Results:
        output(dict) = Parsed output of OpenAIAPI
    """
    if not website:
        website = ""
    llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

    template = """
    I'll give you three inputs. These inputs will be the name of the company, 
    the country of the company, and the website company. The website of the company
    is not mandatory.
    You have to give me the products and services that the company offers as output.
    you dont need to give me nothing more than the ouput.



    input:
    IKEA Deutschland GmbH & Co. KG
    Germany
    ikea.com

    the output must be in this format, please use it:
    "Products/services": Furniture, Home decor, Kitchen and Dining;
    "Keywords":furniture, storage, lighting;
    "Company Classification":5712 (Furniture Stores) – SIC, 442110 (Furniture Stores) – NAICS
    do it yourself now.
    input:
    {name_of_company}
    {country_of_company}
    {website_of_company}

    what is the output?
    """

    prompt = PromptTemplate(
        input_variables=["name_of_company", "country_of_company", "website_of_company"],
        template=template,
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    output = chain.run({"name_of_company": name, 
            "country_of_company": country, 
            "website_of_company":website

    })
    parsed = parse_output(output)

    return parsed


In [29]:
parsed = gpt_call(name="Google", country="United States")
print(parsed)

/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'Products/services': ['Search engine', 'Advertising', 'Cloud computing', 'Software'], 'Keywords': ['Search', 'Advertising', 'Cloud', 'Software', 'Technology'], 'Company Classification': ['7370 (Computer Programming', 'Data Processing', 'and Other Computer Related Services) – SIC', '511210 (Software Publishers) – NAICS']}
{'Products/services': ['Search engine', 'Advertising', 'Cloud computing', 'Software'], 'Keywords': ['Search', 'Advertising', 'Cloud', 'Software', 'Technology'], 'Company Classification': ['7370 (Computer Programming', 'Data Processing', 'and Other Computer Related Services) – SIC', '511210 (Software Publishers) – NAICS']}


In [37]:
products_services_str = " + ".join(parsed["Products/services"])

In [43]:
def google_query_formation(name: str, country: str, products: list) -> str:
    """
    Will manipulate strings to create Google search query

    Args:
        name(str): Name of company
        countr(str): Country of company
        products(list): Products that the company offers

    Return:
        google_query(str): Google query
    """
    products_services_str = " + ".join(products)

    return " + ".join([name, country, products_services_str])

In [12]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.llms import OpenAI
from serpapi import GoogleSearch
class Innoscripta:
    """Class of innoscript solution """
    def __init__(self, name: str, country: str, website: str = None):
        self.name = name
        self.country = country
        if not website:
            self.website = ""
        else:
            self.website = website

    def main(self):
        """
        Will do the innoscripta querying
        """
        parsed_gpt_ouput = self.gpt_call()
        google_query = self.google_query_formation(parsed_gpt_ouput)
        imgs = self.google_search(google_query)
        parsed_gpt_ouput["images"] = imgs

        return parsed_gpt_ouput

    def gpt_call(self) -> dict:
        """
        Will call gpt-3.5-turbo for querying informations about a company.

        Args:
            name(str): Name of the company
            country(str): Country of the company
            website(str): Website of the company

        Results:
            output(dict) = Parsed output of OpenAIAPI
        """
        llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

        prompt = self.prompt_template()

        chain = LLMChain(llm=llm, prompt=prompt)
        output = chain.run({"name_of_company": self.name, 
                "country_of_company": self.country, 
                "website_of_company": self.website

        })
        parsed = self.parse_output(output)
        
        return parsed

    def google_query_formation(self, products: list) -> str:
        """
        Will manipulate strings to create Google search query

        Args:
            name(str): Name of company
            countr(str): Country of company
            products(list): Products that the company offers

        Return:
            google_query(str): Google query
        """
        products_services_str = " + ".join(products)

        return " + ".join([self.name, self.country, products_services_str])

    def google_search(self, query: str) -> list:
        """
        Will query google for images based in the output of OpenAIAPI
        
        Args:
            query(str): Formated query using the output openaiAPI
        
        Results:
            imgs(list): List with URLs for images
        """
        search = GoogleSearch({
        "q": query,
        "engine": "google_images",
        "location": "Austin, Texas",
        "api_key": SERPAPI_KEY
        })
        response = search.get_dict()
        imgs = [r["original"] for r in response["images_results"][:5]]

        return imgs

    def prompt_template(self):
        template = """
        I'll give you three inputs. These inputs will be the name of the company, 
        the country of the company, and the website company. The website of the company
        is not mandatory.
        You have to give me the products and services that the company offers as output.
        you dont need to give me nothing more than the ouput.



        input:
        IKEA Deutschland GmbH & Co. KG
        Germany
        ikea.com

        the output must be in this format, please use it:
        "Products/services": Furniture, Home decor, Kitchen and Dining;
        "Keywords":furniture, storage, lighting;
        "Company Classification":5712 (Furniture Stores) – SIC, 442110 (Furniture Stores) – NAICS
        do it yourself now.
        input:
        {name_of_company}
        {country_of_company}
        {website_of_company}

        what is the output?
        """

        prompt = PromptTemplate(
            input_variables=["name_of_company", "country_of_company", "website_of_company"],
            template=template,
        )

        return prompt

    def parse_output(self, output_langchain: str) -> dict:
        """ 
        Will parse the output_langchain of the Langchain query
        
        Args:
            output(str): The output_langchain of Langchain query.

        Returns:
            result_dict(dict): the parsed output_langchain to dict
        """

        result_dict = {}

        sections = [section.strip() for section in output_langchain.split(";")]

        for section in sections:
            if section:
                header, values_str = section.split(":")
                header = header.strip('"')
                values = [value.strip() for value in values_str.strip('[]').split(",")]

                result_dict[header] = values

        return result_dict

In [13]:
inno = Innoscripta(name="Google", country="United States")

In [14]:
tst = inno.main()

/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/jp/anaconda3/envs/inno/lib/python3.9/site-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [15]:
tst

{'Products/services': ['Search engine',
  'Advertising',
  'Cloud computing',
  'Software'],
 'Keywords': ['Search', 'Advertising', 'Cloud', 'Software', 'Technology'],
 'Company Classification': ['7371 (Computer Programming Services) – SIC',
  '511210 (Software Publishers) – NAICS.'],
 'images': ['https://www.samuraidr.com/wp-content/uploads/2021/09/Screen-Shot-2021-09-01-at-12.43.14-PM-1024x592.png',
  'https://static.semrush.com/blog/uploads/media/ac/63/ac6308fae83bd9c952b0b1a5292d352e/y0zetQ4D7_4Ga5SDq-gSTG9vYjPATpn_bXTRPA7pQ991koJyT9qMWGo-lG8HrXJ1GlHBWd_ld1dzDkpK88VYzI7TBehq61QuHw3HVGAzid65rQjf-ztHcFlvoy657tw4NHa39WB2.png',
  'https://static.semrush.com/blog/uploads/media/e0/3a/e03a2709a8cc1a41f2d66fa2b01a0b44/What-a-Keyword-Is.webp',
  'https://static.semrush.com/blog/uploads/media/58/24/582487daa3d51dc129c42eec007fe665/original.png',
  'https://cdn.shopify.com/s/files/1/0070/7032/files/natural-dog-food.png?format=webp&v=1650031892']}